In [1]:
# !pip install anytree

     |████████████████████████████████| 41 kB 269 kB/s eta 0:00:01


In [3]:
import numpy as np
import pandas as pd

from typing import List, Dict
from anytree import NodeMixin, RenderTree
from anytree.search import find_by_attr, findall_by_attr
from math import log2

pd.set_option('display.max_columns', 20)

### Loading Data

In [4]:
data  = pd.read_csv("data.csv")
df = data.head()
df

,outlook,temp,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes


### Implementing Decision Tree 

In [5]:
class DecTreeNode(NodeMixin):
    def __init__(self, name: str, attribute: str, parent= None):  # type(parent) = DecTreeNode
        super(DecTreeNode, self).__init__()
        self.name = name
        self.attribute = attribute
        self.parent = parent
        self.attr_value = "{0}={1}".format(attribute, name)

In [6]:
class DecTree():
    
    def __init__(self, data: pd.DataFrame, target_attribute: str):
        self.data = data
        self.target_attr = target_attribute
        self.target_attr_vals = data[target_attribute].unique()
        self.root_node = None
        
    def pmf_target(self, df: pd.DataFrame) -> Dict[str, float]:
        # Add your code
        pmf = dict(df[self.target_attr].value_counts()/len(df))
        return pmf
    
    def entropy(self, pmf: Dict[str, float]) -> float:
        # Add your code
        entr = 0
        for k,v in pmf.items():
            if v ==0:
                continue
            entr+= -v*log2(v)
        return entr
    
    def cal_entropy_df(self, df: pd.DataFrame) -> float:
        # Add your code
        pmf = self.pmf_target(df)
        entr = self.entropy(pmf)
        return entr
    
    def info_gain_attribute(self, df: pd.DataFrame, attribute: str) -> float:
        # Add your code
        entropy_children = 0
        entropy_root = self.cal_entropy_df(df)
        for attr in df[attribute].unique():
            len_attr = len(df[df[attribute]==attr])
            entropy_children+= (len_attr/len(df[attribute])) * self.cal_entropy_df(df[df[attribute]==attr])
        information_gain = entropy_root - entropy_children
        return information_gain
    
    def max_info_gain_attribute(self, df: pd.DataFrame, attributes: List[int]) -> str:        
        # Add your code
        info_gains = []
        for attr in attributes:
            info_gains.append(self.info_gain_attribute(df, attr))
        max_gain = max(info_gains) 
        index = info_gains.index(max_gain)
        max_attr = attributes[index]
        return max_attr
    
    def build_tree_infgain(self, df: pd.DataFrame, attr_list: List[str], start_node: DecTreeNode):
        # Add your code
        splitting_attr = self.max_info_gain_attribute(df, attr_list)  # next splitting attr in our dataframe
        children = df[splitting_attr].unique()   # possible values of the splitting attr, we shall split on one of these
  
        for child in children:
#             if len(df[df[splitting_attr] == child][self.target_attr].unique())==1: # return if a stop condition is found
            if self.cal_entropy_df(df[df[splitting_attr] == child]) ==0:
                # print(f"Leaf node of {splitting_attr}={child}: {df[df[splitting_attr] == child][self.target_attr].unique()[0]}")
                self.split_node = DecTreeNode(child, splitting_attr, parent=start_node) # make a new leaf node
                self.split_node = DecTreeNode(df[df[splitting_attr] == child][self.target_attr].unique()[0], "class", parent=self.split_node) # make a new leaf node
            else:
                self.split_node = DecTreeNode(child, splitting_attr, parent=start_node) # else, make a subtree node
                self.build_tree_infgain(df[df[splitting_attr]==child], attr_list, start_node = self.split_node) 
        
        # Use return to end current recursive call when any stopping condition is reached
        return
    
    def generate_tree(self):
        attributes = self.data.columns.to_list()
        attributes.remove(self.target_attr)
        start_node = DecTreeNode("start", "start")
        self.build_tree_infgain(self.data, attributes, start_node)
        
        self.root_node = start_node
    
    def print_tree(self):
        for pre, _, node in RenderTree(self.root_node):
            print(f"{pre} {node.attribute}={node.name}")
            

    def func(self, node):
        for child in node.children:
            if child.attr_value in self.attr_values:
                # print("matched")
                self.func(child)
                return self.predictions
            elif child.name in self.target_attr_vals:
                self.predictions = child.name

    def predict(self, X: pd.DataFrame) -> List[str]:
        # Add your code
        predictions = []
        X_dict = X.to_dict()
        for i in X.index:
            self.attr_values = [(f"{k}={v[i]}")for k,v in X_dict.items()]
            predictions.append(self.func(self.root_node))
        return predictions

### Train Decision Tree 

In [7]:
dec_tree = DecTree(data, "play")

dec_tree.generate_tree()
dec_tree.print_tree()

 start=start
├──  outlook=sunny
│   ├──  humidity=high
│   │   └──  class=no
│   └──  humidity=normal
│       └──  class=yes
├──  outlook=overcast
│   └──  class=yes
└──  outlook=rainy
    ├──  windy=False
    │   └──  class=yes
    └──  windy=True
        └──  class=no


### Perform Inference 

In [8]:
predictions = dec_tree.predict(data)
print(predictions)

['no', 'no', 'yes', 'yes', 'yes', 'no', 'yes', 'no', 'yes', 'yes', 'yes', 'yes', 'yes', 'no']


### Evaluate on the Cars Dataset 

In [9]:
dt_train = pd.read_csv("cars_train.csv")
dt_test = pd.read_csv("cars_test.csv")

dt_test.head()

,buying_price,maintenance_cost,num_doors,num_persons,size_luggage,safety,decision
0,vhigh,med,2,2,small,low,unacc
1,high,high,3,4,big,med,acc
2,low,med,4,2,big,low,unacc
3,vhigh,high,5more,4,small,low,unacc
4,med,med,4,2,med,high,unacc


In [11]:
# Train
dec_tree = DecTree(dt_train, "decision")
dec_tree.generate_tree()
dec_tree.print_tree()

 start=start
├──  safety=high
│   ├──  num_persons=more
│   │   ├──  maintenance_cost=vhigh
│   │   │   ├──  buying_price=med
│   │   │   │   ├──  num_doors=2
│   │   │   │   │   └──  class=unacc
│   │   │   │   ├──  num_doors=5more
│   │   │   │   │   └──  class=acc
│   │   │   │   ├──  num_doors=3
│   │   │   │   │   └──  class=acc
│   │   │   │   └──  num_doors=4
│   │   │   │       └──  class=acc
│   │   │   ├──  buying_price=high
│   │   │   │   └──  class=unacc
│   │   │   ├──  buying_price=low
│   │   │   │   └──  class=acc
│   │   │   └──  buying_price=vhigh
│   │   │       └──  class=unacc
│   │   ├──  maintenance_cost=low
│   │   │   ├──  num_doors=4
│   │   │   │   └──  class=acc
│   │   │   ├──  num_doors=3
│   │   │   │   └──  class=acc
│   │   │   ├──  num_doors=5more
│   │   │   │   └──  class=acc
│   │   │   └──  num_doors=2
│   │   │       ├──  size_luggage=med
│   │   │       │   └──  class=acc
│   │   │       └──  size_luggage=small
│   │   │           └──  class=una

In [23]:
# Test
dt_test_x = dt_test.drop(columns="decision")
dt_test_y = dt_test["decision"].to_list()

preds = dec_tree.predict(dt_test_x)

In [24]:
# Evaluate results
from sklearn.metrics import accuracy_score, confusion_matrix

acc = accuracy_score(dt_test_y, preds)
cm = confusion_matrix(dt_test_y, preds)
print(f"Accuracy: {acc}")
print(cm)

Accuracy: 0.93
[[17  7]
 [ 0 76]]


In [ ]:
def build_tree_infgain(self, df: pd.DataFrame, attr_list: List[str], start_node: DecTreeNode):

    a = self.max_info_gain_attribute(df, attr_list) 
    temp = list(df[a].unique())

    for i in temp:
        if self.cal_entropy_df(df[df[a] == i) ==0:
            self.split_node = DecTreeNode(i, a, parent=start_node) # make a new leaf node
            self.split_node = DecTreeNode(df[df[a] == i][self.target_attr].unique()[0],
            "class", parent=self.split_node) # label the leaf node with class
        else:
            self.split_node = DecTreeNode(i, a, parent=start_node) # else, make a new subtree node
            self.build_tree_infgain(df[df[a]==i], attr_list, start_node = self.split_node) 

    return